In [1]:
# Notes on Using the Google Calendar
# Tony Held tony.held@gmail.com
# Created 02/24/2021

In [2]:
# **********************************************
#    Jupyter Interactive Mode Settings
#
#    These control what is printed in the out: cell
# **********************************************

# How interactive you want is discussed:
# https://ipython.readthedocs.io/en/stable/config/options/terminal.html
# Options are: 'all', 'last', 'last_expr', 'none', 'last_expr_or_assign'
# Default is: 'last_expr'
# InteractiveShell.ast_node_interactivity I can only get last_expr_or_assign to work

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

In [3]:
# **********************************************
# Allows autocomplete to work properly
#
# Tab for auto-complete options, tab to scroll down, return to select
# Shift+Tab to get a functions signature (after you have imported them)
#
# https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working
# **********************************************

%config Completer.use_jedi = False

# Not sure how/when this helps
# %config IPCompleter.greedy = True

In [4]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
# usage pp.pprint(stuff)

In [5]:
def values(var, exclude_starting_with="_"):
    """Show variable attributes and values for diagnostic debugging purposes.
    
    Set exclude_starting_with to '' if you wish to include all attributes."""
    
    print(f'Variable type: {type(var)}')
    
    if exclude_starting_with:
        attrs = [i for i in dir(var) if not i.startswith(exclude_starting_with)]
    else:
        attrs = [i for i in dir(var)]
            
    for i in attrs:
        print(f'\n{i}:\n{"-"*20}\n{getattr(var, i)}')

In [7]:
# Resources
# Source 1: https://www.youtube.com/watch?v=Z5G0luBohCg&list=PLOU2XLYxmsILOIxBRPPhgYbuSslr50KVq&index=4
# Source 2: https://dev.to/karenapp/how-to-get-started-with-google-calendar-api-using-python-with-examples-4c4h
# Source 3: https://github.com/karenapp/google-calendar-python-api/blob/master/list_events.py

In [8]:
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

from datetime import datetime, timedelta

In [12]:
def get_calendar_service():
    """Return a google API service associated with a scope and google oath_key.

    service : googleapiclient.discovery.Resource
    """

    # 1.  Specify one or more scopes (strings or iterable)
    #     that you wish the google user to authorize
    scopes = "https://www.googleapis.com/auth/calendar"

    # 2.  Set name for local file created on google site
    #     which handles API's for registered projects
    google_oath_key = 'client_secret_oath_key_calendar.json'

    # 3.  Local file to store a google user's credential authorization
    store = file.Storage('storage.json')
    credz = store.get()
    if not credz or credz.invalid:
        flow = client.flow_from_clientsecrets(google_oath_key, scopes)
        credz = tools.run_flow(flow, store)

    # 4.  Instantiate Google service with authorization credential
    api = 'calendar'
    version = 'v3'
    service = build(api, version, http=credz.authorize(Http()))
    print(type(service))

    return service

In [13]:
def list_calendars(service):
    print('\nGetting list of calendars\n')
    calendars_result = service.calendarList().list().execute()

    calendars = calendars_result.get('items', [])

    if not calendars:
        print('No calendars found.')
    for calendar in calendars:
        summary = calendar['summary']
        id_ = calendar['id']
        primary = "Primary" if calendar.get('primary') else ""
        # print("%s\t%s\t%s" % (summary, id, primary))
        print(f'{summary}, \t{id_}, \t{primary}')
    print()

    pp.pprint(calendars_result)

In [14]:
def list_events(service):
    print('\nGetting list of events\n')

    # Call the Calendar API
    now = datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    print('Getting List of next 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                        maxResults=10, singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])

    pp.pprint(events)

In [15]:
def update_event(service):
    print('\nUpdating event\n')

    # update the event to tomorrow 9 AM IST

    d = datetime.now().date()
    tomorrow = datetime(d.year, d.month, d.day, 9) + timedelta(days=1)
    start = tomorrow.isoformat()
    end = (tomorrow + timedelta(hours=2)).isoformat()

    event_result = service.events().update(
        calendarId='primary',
        eventId='27mprdt4o3ema2p63ppti2jgrb',
        body={
            "summary": 'Updated Automating calendar',
            "description": 'This is a tutorial example of automating google calendar with python, updated time.',
            "start": {"dateTime": start, "timeZone": 'Asia/Kolkata'},
            "end": {"dateTime": end, "timeZone": 'Asia/Kolkata'},
        },
    ).execute()

    print("updated event")
    print("id: ", event_result['id'])
    print("summary: ", event_result['summary'])
    print("starts at: ", event_result['start']['dateTime'])
    print("ends at: ", event_result['end']['dateTime'])

In [16]:
def scratch01(service):
    """
    service: googleapiclient.discovery.Resource
    """

    print('\nGetting specific calendar\n')

    calendar = service.calendars().get(calendarId='primary').execute()
    print(calendar)
    pp.pprint(calendar)

    sec_cal = 'og1pc1pu59vhkrrsce4ub4llvo@group.calendar.google.com'
    calendar2 = service.calendars().get(calendarId=sec_cal).execute()
    print(calendar2)
    pp.pprint(calendar2)

In [17]:
my_service = get_calendar_service()
print(my_service)
list_calendars(my_service)
list_events(my_service)
update_event(my_service)
scratch01(my_service)

<class 'googleapiclient.discovery.Resource'>

Getting list of calendars

1706.fawn.gate@gmail.com, 	1706.fawn.gate@gmail.com, 	Primary
Holidays in United States, 	en.usa#holiday@group.v.calendar.google.com, 	
Birthdays, 	addressbook#contacts@group.v.calendar.google.com, 	
Secondary Cal, 	og1pc1pu59vhkrrsce4ub4llvo@group.calendar.google.com, 	

{   'etag': '"p324873dhuu1eu0g"',
    'items': [   {   'accessRole': 'owner',
                     'backgroundColor': '#9fe1e7',
                     'colorId': '14',
                     'conferenceProperties': {   'allowedConferenceSolutionTypes': [   'hangoutsMeet']},
                     'defaultReminders': [{'method': 'popup', 'minutes': 30}],
                     'etag': '"1579976561436000"',
                     'foregroundColor': '#000000',
                     'id': '1706.fawn.gate@gmail.com',
                     'kind': 'calendar#calendarListEntry',
                     'notificationSettings': {   'notifications': [   {   'method': 'em

{'kind': 'calendar#calendar', 'etag': '"WjFpr7s39eUXtRI4Ac0v8l8fBaI"', 'id': '1706.fawn.gate@gmail.com', 'summary': '1706.fawn.gate@gmail.com', 'timeZone': 'America/Los_Angeles', 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}
{   'conferenceProperties': {   'allowedConferenceSolutionTypes': [   'hangoutsMeet']},
    'etag': '"WjFpr7s39eUXtRI4Ac0v8l8fBaI"',
    'id': '1706.fawn.gate@gmail.com',
    'kind': 'calendar#calendar',
    'summary': '1706.fawn.gate@gmail.com',
    'timeZone': 'America/Los_Angeles'}
{'kind': 'calendar#calendar', 'etag': '"8a7-KhLBGoEjJyNSW4iHqDsqBVY"', 'id': 'og1pc1pu59vhkrrsce4ub4llvo@group.calendar.google.com', 'summary': 'Secondary Cal', 'description': 'Added by tony2/24/21 at 8:23', 'timeZone': 'America/Los_Angeles', 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}
{   'conferenceProperties': {   'allowedConferenceSolutionTypes': [   'hangoutsMeet']},
    'description': 'Added by tony2/24/21 at 8:23

## Google Calendar Tutorial 1 of 2
http://wescpy.blogspot.com/2015/09/creating-events-in-google-calendar.html

In [20]:
GMT_OFF = '-07:00'    # PDT/MST/GMT-7
EVENT = {
    'summary': 'Games with friends',
    'start':   {'dateTime': '2021-02-25T13:00:00%s' % GMT_OFF},
    'end':     {'dateTime': '2021-02-25T15:00:00%s' % GMT_OFF},
}

{'summary': 'Games with friends',
 'start': {'dateTime': '2021-02-25T13:00:00-07:00'},
 'end': {'dateTime': '2021-02-25T15:00:00-07:00'}}

In [21]:
e = my_service.events().insert(calendarId='primary',
    sendNotifications=True, body=EVENT).execute()

{'kind': 'calendar#event',
 'etag': '"3228381406958000"',
 'id': 'fkvr6vv5art72ohnubtfv7rl3s',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=Zmt2cjZ2djVhcnQ3Mm9obnVidGZ2N3JsM3MgMTcwNi5mYXduLmdhdGVAbQ',
 'created': '2021-02-24T18:18:23.000Z',
 'updated': '2021-02-24T18:18:23.479Z',
 'summary': 'Games with friends',
 'creator': {'email': '1706.fawn.gate@gmail.com', 'self': True},
 'organizer': {'email': '1706.fawn.gate@gmail.com', 'self': True},
 'start': {'dateTime': '2021-02-25T12:00:00-08:00'},
 'end': {'dateTime': '2021-02-25T14:00:00-08:00'},
 'iCalUID': 'fkvr6vv5art72ohnubtfv7rl3s@google.com',
 'sequence': 0,
 'reminders': {'useDefault': True},
 'eventType': 'default'}

In [22]:
print('''*** %r event added:
    Start: %s
    End:   %s''' % (e['summary'].encode('utf-8'),
        e['start']['dateTime'], e['end']['dateTime']))

*** b'Games with friends' event added:
    Start: 2021-02-25T12:00:00-08:00
    End:   2021-02-25T14:00:00-08:00


In [27]:
my_events = my_service.events().list(calendarId='primary').execute()
type(my_events)

dict

In [29]:
pp.pprint(my_events)

{   'accessRole': 'owner',
    'defaultReminders': [{'method': 'popup', 'minutes': 30}],
    'etag': '"p33c93r7ri21uu0g"',
    'items': [   {   'attendees': [   {   'email': 'employee01@tony-held.com',
                                          'responseStatus': 'needsAction'},
                                      {   'email': 'tony.held@gmail.com',
                                          'responseStatus': 'accepted'}],
                     'created': '2021-02-17T19:49:43.000Z',
                     'creator': {   'email': '1706.fawn.gate@gmail.com',
                                    'self': True},
                     'end': {'dateTime': '2021-02-18T22:00:00-08:00'},
                     'etag': '"3227183102458000"',
                     'eventType': 'default',
                     'htmlLink': 'https://www.google.com/calendar/event?eid=YnU0bjhiZDZjNDJkYTBiajhsbTlvYWhmbzggMTcwNi5mYXduLmdhdGVAbQ',
                     'iCalUID': 'bu4n8bd6c42da0bj8lm9oahfo8@google.com',
             

In [31]:
for k, v in my_events.items():
    if k != 'items':
        print(f'{k}: {v}')

kind: calendar#events
etag: "p33c93r7ri21uu0g"
summary: 1706.fawn.gate@gmail.com
updated: 2021-02-24T18:18:23.479Z
timeZone: America/Los_Angeles
accessRole: owner
defaultReminders: [{'method': 'popup', 'minutes': 30}]
nextSyncToken: CNiR7PuQg-8CENiR7PuQg-8CGAUg4umFqwE=


In [34]:
for i, ev in enumerate(my_events['items']):
    print(f'Item # {i}')
    pp.pprint(ev)
    print(f'\n')
    

Item # 0
{   'attendees': [   {   'email': 'employee01@tony-held.com',
                         'responseStatus': 'needsAction'},
                     {   'email': 'tony.held@gmail.com',
                         'responseStatus': 'accepted'}],
    'created': '2021-02-17T19:49:43.000Z',
    'creator': {'email': '1706.fawn.gate@gmail.com', 'self': True},
    'end': {'dateTime': '2021-02-18T22:00:00-08:00'},
    'etag': '"3227183102458000"',
    'eventType': 'default',
    'htmlLink': 'https://www.google.com/calendar/event?eid=YnU0bjhiZDZjNDJkYTBiajhsbTlvYWhmbzggMTcwNi5mYXduLmdhdGVAbQ',
    'iCalUID': 'bu4n8bd6c42da0bj8lm9oahfo8@google.com',
    'id': 'bu4n8bd6c42da0bj8lm9oahfo8',
    'kind': 'calendar#event',
    'organizer': {'email': '1706.fawn.gate@gmail.com', 'self': True},
    'reminders': {'useDefault': True},
    'sequence': 0,
    'start': {'dateTime': '2021-02-18T19:00:00-08:00'},
    'status': 'confirmed',
    'summary': 'Dinner with friends',
    'updated': '2021-02-17T19:52:3

In [36]:
e_id = 'fkvr6vv5art72ohnubtfv7rl3s'
event1 = my_service.events().get(calendarId='primary', eventId=e_id).execute()
pp.pprint(event1)

{   'created': '2021-02-24T18:18:23.000Z',
    'creator': {'email': '1706.fawn.gate@gmail.com', 'self': True},
    'end': {'dateTime': '2021-02-25T14:00:00-08:00'},
    'etag': '"3228381406958000"',
    'eventType': 'default',
    'htmlLink': 'https://www.google.com/calendar/event?eid=Zmt2cjZ2djVhcnQ3Mm9obnVidGZ2N3JsM3MgMTcwNi5mYXduLmdhdGVAbQ',
    'iCalUID': 'fkvr6vv5art72ohnubtfv7rl3s@google.com',
    'id': 'fkvr6vv5art72ohnubtfv7rl3s',
    'kind': 'calendar#event',
    'organizer': {'email': '1706.fawn.gate@gmail.com', 'self': True},
    'reminders': {'useDefault': True},
    'sequence': 0,
    'start': {'dateTime': '2021-02-25T12:00:00-08:00'},
    'status': 'confirmed',
    'summary': 'Games with friends',
    'updated': '2021-02-24T18:18:23.479Z'}


In [41]:
TIMEZONE = 'America/Los_Angeles'
EVENT = {
    'summary': 'Games with friends via patch',
    'start':  {'dateTime': '2021-02-26T19:00:00', 'timeZone': TIMEZONE},
    'end':    {'dateTime': '2021-02-26T22:00:00', 'timeZone': TIMEZONE},
    'recurrence': ['RRULE:FREQ=MONTHLY;INTERVAL=2;UNTIL=20171231']
}

{'summary': 'Games with friends via patch',
 'start': {'dateTime': '2021-02-26T19:00:00',
  'timeZone': 'America/Los_Angeles'},
 'end': {'dateTime': '2021-02-26T22:00:00', 'timeZone': 'America/Los_Angeles'},
 'recurrence': ['RRULE:FREQ=MONTHLY;INTERVAL=2;UNTIL=20171231']}

In [42]:
event2 = my_service.events().patch(calendarId='primary', eventId=e_id, body=EVENT).execute()
pp.pprint(event2)

{   'created': '2021-02-24T18:18:23.000Z',
    'creator': {'email': '1706.fawn.gate@gmail.com', 'self': True},
    'end': {   'dateTime': '2021-02-26T22:00:00-08:00',
               'timeZone': 'America/Los_Angeles'},
    'etag': '"3228392790800000"',
    'eventType': 'default',
    'htmlLink': 'https://www.google.com/calendar/event?eid=Zmt2cjZ2djVhcnQ3Mm9obnVidGZ2N3JsM3NfMjAyMTAyMjdUMDMwMDAwWiAxNzA2LmZhd24uZ2F0ZUBt',
    'iCalUID': 'fkvr6vv5art72ohnubtfv7rl3s@google.com',
    'id': 'fkvr6vv5art72ohnubtfv7rl3s',
    'kind': 'calendar#event',
    'organizer': {'email': '1706.fawn.gate@gmail.com', 'self': True},
    'recurrence': ['RRULE:FREQ=MONTHLY;UNTIL=20171231;INTERVAL=2'],
    'reminders': {'useDefault': True},
    'sequence': 2,
    'start': {   'dateTime': '2021-02-26T19:00:00-08:00',
                 'timeZone': 'America/Los_Angeles'},
    'status': 'confirmed',
    'summary': 'Games with friends via patch',
    'updated': '2021-02-24T19:53:15.400Z'}


In [43]:
TIMEZONE = 'America/Los_Angeles'
EVENT = {
    'summary': 'Games with friends via update',
    'start':  {'dateTime': '2021-02-27T19:00:00', 'timeZone': TIMEZONE},
    'end':    {'dateTime': '2021-02-27T22:00:00', 'timeZone': TIMEZONE},
    'recurrence': ['RRULE:FREQ=MONTHLY;INTERVAL=2;UNTIL=20171231']
}

{'summary': 'Games with friends via update',
 'start': {'dateTime': '2021-02-27T19:00:00',
  'timeZone': 'America/Los_Angeles'},
 'end': {'dateTime': '2021-02-27T22:00:00', 'timeZone': 'America/Los_Angeles'},
 'recurrence': ['RRULE:FREQ=MONTHLY;INTERVAL=2;UNTIL=20171231']}

In [44]:
event3 = my_service.events().update(calendarId='primary', eventId=e_id, body=EVENT).execute()
pp.pprint(event3)

{   'created': '2021-02-24T18:18:23.000Z',
    'creator': {'email': '1706.fawn.gate@gmail.com', 'self': True},
    'end': {   'dateTime': '2021-02-27T22:00:00-08:00',
               'timeZone': 'America/Los_Angeles'},
    'etag': '"3228392903850000"',
    'eventType': 'default',
    'htmlLink': 'https://www.google.com/calendar/event?eid=Zmt2cjZ2djVhcnQ3Mm9obnVidGZ2N3JsM3NfMjAyMTAyMjhUMDMwMDAwWiAxNzA2LmZhd24uZ2F0ZUBt',
    'iCalUID': 'fkvr6vv5art72ohnubtfv7rl3s@google.com',
    'id': 'fkvr6vv5art72ohnubtfv7rl3s',
    'kind': 'calendar#event',
    'organizer': {'email': '1706.fawn.gate@gmail.com', 'self': True},
    'recurrence': ['RRULE:FREQ=MONTHLY;UNTIL=20171231;INTERVAL=2'],
    'reminders': {'useDefault': True},
    'sequence': 3,
    'start': {   'dateTime': '2021-02-27T19:00:00-08:00',
                 'timeZone': 'America/Los_Angeles'},
    'status': 'confirmed',
    'summary': 'Games with friends via update',
    'updated': '2021-02-24T19:54:11.925Z'}
